# tbcalc - Calculate resolution functions of toroidally bent crystal analysers

This notebook demostrates the use of `Analyser`, the main class of _tbcalc_ package to calculate the reflectivity curves of toroidally bent crystals.

In [2]:
import sys
import os.path

import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(1, '..')
from tbcalc import Analyser, Quantity

(The imported `Quantity` is actually from _PyTTE_ but for convenience it can be imported via _tbcalc_ as well.)

## Defining the TBCA

The characteristics of a TBCA can be passed either as keyword arguments or in a text file. The initialization of `Analyser` utilizes is built on the top of `TTcrystal` class of _pyTTE_. The mandatory keywords required by `TTcrystal` are `crystal`, `hkl`, and `thickness`. Also optional parameters of `TTcrystal` are supported (except for `fix_to_axes`).

In addition, `Analyser` needs the meridional and sagittal bending radii `Rx` and `Ry` (or `R` for spherical bending) and the following keywords to determine the geometry of the crystal wafer:

- **Circular analyser**  `diameter`
- **Rectangular analyser** `a`, `b`
- **Strip-bent analyser** `diameter`, `strip_width`, `strip_orientation` (optional)

In the following we define a spherically bent crystal analyser with the minimum number of parameters:

In [3]:
kwargs = {
          'crystal'   : 'Si',
          'hkl'       : [6,6,0],
          'thickness' : Quantity(150,'um'),
          'R'         : Quantity(1, 'm'),
          'diameter'  : Quantity(100,'mm'),
}

analyser = Analyser(**kwargs)

Note that for parameters with physical dimensions, `Quantity` class is used to avoid manual unit conversion errors.